# PyKeops

This notebook gathers some examples available on the Keops website (https://www.kernel-operations.io/keops/_auto_examples/index.html#)

The notebook is organized as:
- I/ installation procedure
- II/ First example to play with the LazyTensors
- III/ Second example to play with the LazyTensors
- Then three applications are presented (k-NN on MNIST, kernel interpolation, k-means). Feel free to play with them depending on your interest.

## Installation

Don't forget to swith GPU accelerator on: **Runtime>Change runtime type>Hardware accelerator** to GPU

In [ ]:
!pip install pykeops[colab]

## First example with LazyTensors

### Straight to Keops

In [ ]:
import torch

N,M,D = 10**5, 10**5, 10
x = torch.rand(N,D).cuda()
y = torch.rand(M,D).cuda()

# turning Tensors into Keops symbolic variables
from pykeops.torch import LazyTensor
import time
x_i = LazyTensor(x[:,None,:]) # x_i.shape = (N,1,D)
y_j = LazyTensor(y[None,:,:]) # y_j.shape = (1,M,D)
D_ij = ((x_i-y_j)**2).sum(dim=2) # symbolic (N,M,1) matrix of squarred distances
# Note that nothing has been computed yet, everything will be done in the final reduction step

# first time called triggers the compilation + Warming up GPU
indices_i = D_ij.argmin(dim=1)

start = time.perf_counter()
indices_i = D_ij.argmin(dim=1) 
torch.cuda.synchronize()
end = time.perf_counter()

print(end-start)
print(s_i[:5])


### Comparison with pytorch

In [ ]:
x_i = x[:, None, :]  # (M, 1, D) torch array
y_j = y[None, :, :]  # (1, N, D) torch array

start = time.perf_counter()
D_ij = ((x_i - y_j) ** 2).sum(-1)  # (M, N) array of squared distances |x_i-y_j|^2
s_i = D_ij.argmin(dim=1)  # (M,)   array of integer indices
torch.cuda.synchronize()

end = time.perf_counter()
print(end-start)
print(s_i[:5])

Fell free to play with M,N,D to see memory overflow and timing differences !

## Second example

In [ ]:
M,N = (100000,200000)
D = 3
x = torch.randn(M, D).cuda()  # M target points in dimension D, stored on the GPU
y = torch.randn(N, D).cuda()  # N source points in dimension D, stored on the GPU
b = torch.randn(N, 4).cuda()  # N values of the 4D source signal, stored on the GPU

# x.requires_grad = True  # In the next section, we'll compute gradients wrt. x!

from pykeops.torch import Vi, Vj # nice utilities

x_i = Vi(x) # (M, 1, D) LazyTensor, equivalent to LazyTensor( x[:,None,:] )
y_j = Vj(y) # (1, N, D) LazyTensor, equivalent to LazyTensor(y[None, :, :])  
b_j = Vj(b) # (1, N, D) LazyTensor, equivalent to LazyTensor(b[None, :, :])  
sigma = 0.5

D_ij = ((x_i - y_j) ** 2 / sigma**2).sum(-1) # symbolic (M,N) matrix
K_ij = (-D_ij).exp()  # Symbolic (M, N) Gaussian kernel

a_i = K_ij @ b  # arming up and compilation

start = time.perf_counter()
a_i = K_ij @ b  # The matrix-vector product "@" can be used on "raw" PyTorch tensors!
torch.cuda.synchronize()
end = time.perf_counter()

print(end-start)
print("a_i is now a {} of shape {}.".format(type(a_i), a_i.shape))

## k-nn on MNIST

Classification of digits using nearest neighbor search

Loading MNIST database: 70,000 images (28,28) with labels

In [ ]:
from pykeops.torch import LazyTensor, Vi, Vj
import time

import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

mnist = fetch_openml("mnist_784", cache=False)

x = torch.tensor(mnist.data.astype("float32"))
y = torch.tensor(mnist.target.astype("int64"))

Split train and test set:

In [ ]:
D = x.shape[1]
Ntrain, Ntest = (60000, 10000)
# using continuous is crucial for keops. It raise an error if the data is not
x_train, y_train = x[:Ntrain, :].contiguous(), y[:Ntrain].contiguous()
x_test, y_test = (
    x[Ntrain : Ntrain + Ntest, :].contiguous(),
    y[Ntrain : Ntrain + Ntest].contiguous(),
)

Perform the K-NN classification on 10,000 test images in dimension 784:

In [ ]:
K = 3  # N.B.: K has very little impact on the running time


X_i = Vi(x_test) # (10000, 1, 784) test set
X_j = Vj(x_train)  # (1, 60000, 784) train set
D_ij = ((X_i - X_j) ** 2).sum(-1)  # (10000, 60000) symbolic matrix 

# compilation
ind_knn = D_ij.argKmin(K, dim=1) 

start = time.perf_counter()
ind_knn = D_ij.argKmin(K, dim=1)  # Samples <-> Dataset, (N_test, K)
lab_knn = y_train[ind_knn]  # (N_test, K) array of integers in [0,9]
y_knn, _ = lab_knn.mode()  # Compute the most likely label
end = time.perf_counter()

error = (y_knn != y_test).float().mean().item()

print(
    "{}-NN on the full MNIST dataset: test error = {:.2f}% in {:.2f}s.".format(
        K, error * 100, end-start
    )
)



Display

In [ ]:
plt.figure(figsize=(12, 8))
for i in range(6):
    ax = plt.subplot(2, 3, i + 1)
    ax.imshow((255 - x_test[i]).view(28, 28).detach().cpu().numpy(), cmap="gray")
    ax.set_title("label = {}".format(y_knn[i].int()))
    plt.axis("off")

plt.show()

## Kernel interpolation

Sovling $a^* = \underset{a}{\textrm{argmin}} \| (\alpha Id + K)a -b \|_2^2$
where $K$ is a symetric definite linear opeartor and $\alpha > 0$

Generate some data:

In [3]:
import torch
from pykeops.torch import LazyTensor, Vi, Vj
import time
import matplotlib.pyplot as plt

use_cuda = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

N = 10000
# Sampling locations:
x = torch.rand(N, 2).type(dtype)

# Some random-ish 2D signal:
b = ((x - 0.5) ** 2).sum(1, keepdim=True)
b[b > 0.4 ** 2] = 0
b[b < 0.3 ** 2] = 0
b[b >= 0.3 ** 2] = 1
b = b + 0.05 * torch.randn(N, 1).type(dtype)

# Add 25% of outliers:
Nout = N // 4
b[-Nout:] = torch.rand(Nout, 1).type(dtype)


Define our regression model: Laplacian kernel of deviation sigma

In [4]:
def laplacian_kernel(x, y, sigma=0.1):
    x_i = LazyTensor(x[:, None, :])  # (M, 1, 1)
    y_j = LazyTensor(y[None, :, :])  # (1, N, 1)
    D_ij = ((x_i - y_j) ** 2).sum(-1)  # (M, N) symbolic matrix of squared distances
    return (-D_ij.sqrt() / sigma).exp()  # (M, N) symbolic Laplacian kernel matrix


Perform the kernel interpolation using the solve routine

In [ ]:
alpha = 10  # Ridge regularization

start = time.time()

K = laplacian_kernel(x, x)
a = K.solve(b, alpha=alpha) # remember first time call for compilation

end = time.time()

print(
    "Time to perform an RBF interpolation with {:,} samples in 2D: {:.5f}s".format(
        N, end - start
    )
)

In [ ]:
# Extrapolate on a uniform sample:
X = Y = torch.linspace(0, 1, 101).type(dtype)
X, Y = torch.meshgrid(X, Y)
t = torch.stack((X.contiguous().view(-1), Y.contiguous().view(-1)), dim=1)

K_tx = laplacian_kernel(t, x)
mean_t = K_tx @ a
mean_t = mean_t.view(101, 101)

# 2D plot: noisy samples and interpolation in the background
plt.figure(figsize=(8, 8))

plt.scatter(
    x.cpu()[:, 0], x.cpu()[:, 1], c=b.cpu().view(-1), s=25000 / len(x), cmap="bwr"
)
plt.imshow(
    mean_t.cpu().numpy()[::-1, :],
    interpolation="bilinear",
    extent=[0, 1, 0, 1],
    cmap="coolwarm",
)

# sphinx_gallery_thumbnail_number = 2
plt.axis([0, 1, 0, 1])
plt.tight_layout()
plt.show()



## K-means algorithm

Goal: use the bruteforce NN search to implement a large-scale K-means clustering wo/ memory overflow


In [8]:
import time
import torch
from matplotlib import pyplot as plt
from pykeops.torch import LazyTensor

use_cuda = torch.cuda.is_available()
dtype = torch.float32 if use_cuda else torch.float64

In [9]:
def KMeans(x, K=10, Niter=10, verbose=True):
    """Implements Lloyd's algorithm for the Euclidean metric."""

    start = time.time()
    N, D = x.shape  # Number of samples, dimension of the ambient space

    c = x[:K, :].clone()  # Simplistic initialization for the centroids

    x_i = LazyTensor(x.view(N, 1, D))  # (N, 1, D) samples
    c_j = LazyTensor(c.view(1, K, D))  # (1, K, D) centroids

    # K-means loop:
    # - x  is the (N, D) point cloud,
    # - cl is the (N,) vector of class labels
    # - c  is the (K, D) cloud of cluster centroids
    for i in range(Niter):

        # E step: assign points to the closest cluster -------------------------
        D_ij = ((x_i - c_j) ** 2).sum(-1)  # (N, K) symbolic squared distances
        cl = D_ij.argmin(dim=1).long().view(-1)  # Points -> Nearest cluster

        # M step: update the centroids to the normalized cluster average: ------
        # Compute the sum of points per cluster:
        c.zero_()
        c.scatter_add_(0, cl[:, None].repeat(1, D), x)

        # Divide by the number of points per cluster:
        Ncl = torch.bincount(cl, minlength=K).type_as(c).view(K, 1)
        c /= Ncl  # in-place division to compute the average

    if verbose:  # Fancy display -----------------------------------------------
        if use_cuda:
            torch.cuda.synchronize()
        end = time.time()
        print(
            f"K-means for the Euclidean metric with {N:,} points in dimension {D:,}, K = {K:,}:"
        )
        print(
            "Timing for {} iterations: {:.5f}s = {} x {:.5f}s\n".format(
                Niter, end - start, Niter, (end - start) / Niter
            )
        )

    return cl, c

First xp

In [ ]:
N, D, K = 10000, 2, 50
x = 0.7 * torch.randn(N, D, dtype=dtype) + 0.3
cl, c = KMeans(x, K)

plt.figure(figsize=(8, 8))
plt.scatter(x[:, 0].cpu(), x[:, 1].cpu(), c=cl.cpu(), s=30000 / len(x), cmap="tab10")
plt.scatter(c[:, 0].cpu(), c[:, 1].cpu(), c="black", s=50, alpha=0.8)
plt.axis([-2, 2, -2, 2])
plt.tight_layout()
plt.show()

Second cp: N=10^6 in D=100 w/ K=1,000 classes

In [ ]:
N, D, K = 1000000, 100, 1000
x = torch.randn(N, D, dtype=dtype)
cl, c = KMeans(x, K)